In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#https://developers.generativeai.google/api/python/google/generativeai
!pip install google-generativeai

In [3]:
import re
import time
import google.generativeai as palm
import pandas as pd
import pandas as pd
import os
palm.configure(api_key="api key here")

# Chnage working directory

In [4]:
%cd "/content/drive/MyDrive/ABSA work/codes/bard/"

/content/drive/MyDrive/ABSA work/codes/bard


In [5]:
%pwd

'/content/drive/MyDrive/ABSA work/codes/bard'

In [6]:
defaults = {
  'model': 'models/text-bison-001',
  'temperature': 0.2,
  'candidate_count': 1,
  'top_k': 40,
  'top_p': 0.95,
  'max_output_tokens': 1024,
  'stop_sequences': [],
  'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
}

# Set path for saving the results

In [7]:
path = "results/YASO/"
filename="TSA-MD_val"

# Load Dataset

In [14]:
df = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_YASO/"+filename+".csv")
df = df.dropna(axis=1, how='all')
print(len(df))
df.head(10)

191


,sentence,aspect_terms,polarity
0,"""Typical"" Italian food was far from typical.",['Italian food'],['negative']
1,After a few weeks the tablet started asking fo...,"['sound', 'buy', 'wireless headphones', 'resol...","['positive', 'positive', 'positive', 'positive']"
2,All the doctors and staff present were attenti...,['doctors and staff present'],['positive']
3,"Also, the fuel injection system does not work ...",['the fuel injection system'],['negative']
4,Always seems to understand me with perfectly c...,"['clarity', 'commands']","['positive', 'positive']"
5,And great performance with hard work.,['performance'],['positive']
6,"As a result , I got bacteria in my stomach and...",['bacteria in my stomach'],['negative']
7,Banks are entities that help us to carry out p...,['projects'],['positive']
8,Built in webcam works fantastically well.,['Built in webcam'],['positive']
9,But I had to wait a long time.,['wait'],['negative']


In [9]:

if(os.path.isfile(path+filename+"_bard.csv")):
    temp = pd.read_csv(path+filename+"_bard.csv")
else:
    temp=[]
print(len(temp))


190


In [15]:
print(len(df))

191


In [20]:
df.tail(30)

,sentence,aspect_terms,polarity
161,this service is great the speed of this inter...,"['speed of this internet', 'service']","['positive', 'positive']"
162,very good food in this restaurant and the atte...,"['attention', 'food', 'restaurant']","['positive', 'positive', 'positive']"
163,very pleasant place to spend some days near th...,"['cost', 'place', 'attention']","['positive', 'positive', 'positive']"
164,"we stayed 5 days in the hotel, the wonderful f...","['food', 'pool', 'hotel']","['positive', 'positive', 'positive']"
165,when I bought this phone I thought it has 128g...,"['system', 'phone']","['negative', 'negative']"
166,with this product I have enjoyed working in ap...,['product'],['positive']
167,"yesterday, I ordered four chicken sandwiches b...","['delivered', 'quality of food', 'chicken sand...","['negative', 'negative', 'negative']"
168,you can find there anything you want .,['there'],['positive']
169,you can let your child play in the kids area ...,['kids area'],['positive']
170,I started reading it but never laughed.,"['kids area', 'Target Not Found']","['positive', 'Not Available']"


In [16]:
def predict_sentiment(input, aspects):
## joint task this prompt will find out the aspect and sentiments at a same time
#     prompt = f"""As an aspect-based sentiment analyzer, your task is to extract aspects and corresponding sentiments (Positive, Negative, or Neutral) from people's reviews. Each review consists of a text and may contain multiple aspects and their corresponding sentiments. Your goal is to process the reviews and extract the aspects and sentiments from them.

# Please analyze the provided review below and present the aspects and sentiments in the specified format. Ensure that the aspects you provide show some relation and limit them to the most important aspects in the review. Please do not explain any aspect or sentiment. I do not need any extra text. Just the lists of aspects and the sentiments.

# Review: {input}

# Required output Format:

# aspect: ['aspect1', 'aspect2', 'aspect3', ...]
# Sentiment: ['sentiment1', 'sentiment2', 'sentiment3', ..]"""
    prompt = f"""As an aspect-based sentiment analyzer, your task is to extract sentiments (Positive, Negative, or Neutral) from people's review for a given list of aspects.
    Each review consists of a text and may contain single or multiple aspects and their corresponding sentiments. You will be provided with the review and the list of aspects.
    Your goal is to process the reviews and given list of aspects. Then, predict sentiment of each given aspect.

Please present the result in specified format.
Please do not explain any aspect or sentiment. I do not need any extra text. Just the lists of sentiments.

Review: {input}
Aspects: {aspects}
Required output Format:
Sentiment: ['sentiment1', 'sentiment2', 'sentiment3', ..]"""


    response = palm.generate_text(
      **defaults,
      prompt=prompt
    )
    print(response.result)
    return response.result.lower()

In [12]:
feedback=[]
keywords= ['sentiment']
results = {rubric: [] for rubric in keywords}
for i in range(len(temp), len(df)):
  data = df[i:i+1]
  for index, row in data.iterrows():
    # Get the text to assess
    input = row["sentence"]
    # aspects = row["aspect_categories"]
    aspects = row["aspect_terms"]

    print(input, aspects)
    try:
      response = predict_sentiment(input, aspects)
    except:
      response=""
    # print(type(response))
    feedback.append(str(response))
    # print(len(feedback))
    for i in range(len(keywords)):
        keyword = keywords[i]
        pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
        match = pattern.search(response)
        if match:
            results[keyword].append(match.group(1).strip()[2:].lower())
        else:
            results[keyword].append(response)
  # data['Response_bard'] = feedback
  data.loc[:, "Response_bard"] = feedback
  print(results['sentiment'])
  for keyword in keywords:
    data.loc[:, keyword+"_bard"] = results[keyword]
  print(response)
  print(row)
  data.to_csv(path+filename+"_bard.csv", index = False, header=False, mode='a')
  # data.to_csv(path+filename+"_bard.csv", index = False)
  results = {rubric: [] for rubric in keywords}
  feedback=[]
  time.sleep(10)

It was not what I expected. ['kids area', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found', 'Target Not Found']
['Negative']
["['negative']"]
['negative']
sentence                              It was not what I expected.
aspect_terms    ['kids area', 'Target Not Found', 'Target Not ...
polarity        ['positive', 'Not Available', 'Not Available',...
Name: 190, dtype: object


<ipython-input-12-7db29a4674c9>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Response_bard"] = feedback
<ipython-input-12-7db29a4674c9>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, keyword+"_bard"] = results[keyword]


In [17]:
tem= pd.read_csv(path+filename+"_bard.csv")
print(len(tem))
tem.(50)

191


,"""Typical"" Italian food was far from typical.",['Italian food'],['negative'],sentiment: ['negative'],['negative'].1
0,After a few weeks the tablet started asking fo...,"['sound', 'buy', 'wireless headphones', 'resol...","['positive', 'positive', 'positive', 'positive']","sentiment: ['okay', 'positive', 'positive', 'p...","['okay', 'positive', 'positive', 'positive']"
1,All the doctors and staff present were attenti...,['doctors and staff present'],['positive'],['positive'],['positive']
2,"Also, the fuel injection system does not work ...",['the fuel injection system'],['negative'],sentiment: ['negative'],['negative']
3,Always seems to understand me with perfectly c...,"['clarity', 'commands']","['positive', 'positive']","sentiment: ['positive', 'positive']","['positive', 'positive']"
4,And great performance with hard work.,['performance'],['positive'],['positive'],['positive']
5,"As a result , I got bacteria in my stomach and...",['bacteria in my stomach'],['negative'],['negative'],['negative']
6,Banks are entities that help us to carry out p...,['projects'],['positive'],sentiment: ['positive'],['positive']
7,Built in webcam works fantastically well.,['Built in webcam'],['positive'],['positive'],['positive']
8,But I had to wait a long time.,['wait'],['negative'],['negative'],['negative']
9,"But what an annoying bank, a lot of requiremen...","['attention', 'bank']","['negative', 'negative']","sentiment: ['negative', 'negative']","['negative', 'negative']"
